## Pandas analysis

In the following a series of exercises is proposed on a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGA's. Each measurement (i.e. each raw) consists of the address of the TDC providing the signal, 'FPGA' and 'TDC_Channel, and the timing information itself, 'ORBIT_CNT', 'BX_COUNTER' and 'TDC_MEAS'. Each TDC count correspond 25/30 ns, whereas the BX_COUNTER feauters gets updated every 25 ns and the ORBIT_CNT every 'x' BX_COUNTER. You can see these way of storing the time as similar to hours, minutes and seconds.

1\. Create a Pandas DataFrame by read N raws of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of raws and larger that 10k.

2\. Find out the value of 'x'

3\. Find out how much the data taking lasted. You can either make an estimate on the baseis of the fraction of the measurements (raws) you read, or perform this check precisely by reading out the whole dataset

4\. Create a new column with the actual time in ns (as a combination of the other three columns with timing information)

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1

6\. Create a new DataFrame with only the raws with HEAD=1

7\. Make two occupancy plots (one per FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# QUESTION 1
filePath = "/home/agostini/data/data_000637.txt"
N_rows = 2000000
# check if number of rows is greater than data
if N_rows > 1310720: N_rows = 1310720
if N_rows < 10000  : N_rows =   10000
# read file
data = pd.read_csv(filePath, nrows=N_rows)

# QUESTION 2
# in order to find 'x', I'll check for the max+1 in
# BX_COUNTER column, because the probability to never
# find the value in ~10000000 measurements is near to 0
x = data["BX_COUNTER"].max()+1
print("'x' =", x)

# QUESTIONS 3-4
data['t_[ns]']=data['TDC_MEAS']*25/30+data['BX_COUNTER']*25+data['ORBIT_CNT']*x*25
# starting time
t0 = data['t_[ns]'].min()
data['t-t0_[ns]']=data['TDC_MEAS']*25/30+data['BX_COUNTER']*25+data['ORBIT_CNT']*x*25 - t0
data_taking = data['t-t0_[ns]'].max() - data['t-t0_[ns]'].min()
print("Data taking for "+str(N_rows)+" aquisitions [ns] =", data_taking)

# QUESTION 5
np.random.seed(20112018)
data['HEAD'] = np.random.randint(0, 2, len(data['HEAD']))

# QUESTION 6
new_data = data[data['HEAD'] == 1]


'x' = 3564
Data taking for 1310720 aquisitions [ns] = 980416253.312


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,t_[ns],t-t0_[ns]
0,1,0,123,3869200167,2374,26,3.447457e+14,0.000000e+00
3,1,0,64,3869200167,2558,19,3.447457e+14,4.594125e+03
4,1,0,64,3869200167,2760,25,3.447457e+14,9.649125e+03
9,1,0,60,3869200167,2788,7,3.447457e+14,1.033412e+04
10,1,1,7,3869200167,2785,4,3.447457e+14,1.025662e+04
11,1,0,64,3869200167,2786,19,3.447457e+14,1.029412e+04
12,1,1,6,3869200167,2792,18,3.447457e+14,1.044331e+04
13,1,0,36,3869200167,2791,23,3.447457e+14,1.042250e+04
15,1,1,139,3869200167,2797,0,3.447457e+14,1.055331e+04
18,1,1,5,3869200167,2795,4,3.447457e+14,1.050662e+04
